## Here I want to do several things
The first routine should take a given a set of points in 3D and compute the list of all possible tetrahedrons.

The second routine should take a second set of 3D points and the list of all tetrahedrons computed before and computes their barycentric coordinates for the tetrahedrons containing them.

I did refresh my memory by reading this http://www.iue.tuwien.ac.at/phd/nentchev/node31.html

In [1]:
from scipy.spatial import ConvexHull
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import itertools
import scipy.spatial

### List of all possible tethrahedrons
The first routine should take a given a set of points in 3D and compute the list of all possible tetrahedrons.

In [14]:
#generate some random test points
N_A = 10 # like 10 points...
set_3D_points_A = np.random.rand(N_A,3) # all values between [0,1]

print set_3D_points_A[0:3,:], np.shape(set_3D_points_A) # only show the first 3 points

[[ 0.78291039  0.14393185  0.75492195]
 [ 0.99516805  0.56603119  0.4872887 ]
 [ 0.85381135  0.526314    0.48443321]] (10, 3)


We start with a definition, a tetrahedron is made of four points/vertex in 3D spaces, it has four triangle faces.

So if I have *N* points, then all possible combinations of *4* of them give me the list of possible tetrahedrons.

The function below tries to do so using all possible combinations.

In [15]:
def fun_list_all_tetrahedron_from_set_of_points(set_3D_points):
    """
    The function does what its name says.
    It takes a list of a least four points in 3D space and return
    all possible combinations of four points defining a tetrahedron.
    """
    # get the list of indexs for each tetrahedron
    N = np.shape(set_3D_points)[0]
    vec_N = np.arange(0,N)
    
    # get the coordinates for each list of tetrahedra 
    nb_combibation = itertools.combinations(vec_N, 4)
    list_combination = np.array(list(nb_combibation))
    #tetrahedron_coordinates = set_3D_points[nb_combibation,:]
    return list_combination


**nb_combibation** gives the index numbers for each group of four points defining a tetrahedron.

This approach didn't test if for a set of four points the points were in the same plan and didn't tell if the tetrahedrons were exclusive or not in the sens no points were located within a tetrahedron. You may want to know how many unique tetrahedron there are in your set of points.

Some test data below to try the function.

In [16]:
list_index_tetra = fun_list_all_tetrahedron_from_set_of_points(set_3D_points_A)
print len(list_index_tetra),list_index_tetra[0:5,:], np.shape(list_index_tetra)
print "For a set of %1.0f points we have %1.0f tetrahedronds" % (N_A,np.shape(list_index_tetra)[0])

210 [[0 1 2 3]
 [0 1 2 4]
 [0 1 2 5]
 [0 1 2 6]
 [0 1 2 7]] (210, 4)
For a set of 10 points we have 210 tetrahedronds


## Compute the barycentric coordinates for the tetrahedrons
The second routine should take a second set of 3D points and the list of all tetrahedrons computed before and computes their barycentric coordinates for the tetrahedrons containg them.

In [17]:
N_B = 20
set_3D_points_B = np.random.rand(N_B,3) # all values between [0,1]
set_3D_points_B = np.vstack([set_3D_points_B, [0.5, 0.5, 0.5]])
print set_3D_points_B[-3:,:]

[[ 0.30915753  0.82213029  0.36098252]
 [ 0.96040861  0.92827292  0.67253595]
 [ 0.5         0.5         0.5       ]]


I basically use the scipy spatial function here... What I do is the Delaunay triangulation to *clean* the number of good tetrahedrons. It also makes it easier to use the function from scipy.

In [18]:
def fun_compute_barycentric_coordinates(tetrahedron_points, targets_points):
    """
    The function take one tetrahedron as input and a list of points.
    For each point we want to know if it is located in the this tetrahedron, 
    and if yes we want the barycentric coordinates.
    """
    res_bcoords = np.zeros((np.shape(targets_points)[0],4))
    #print np.shape(res_bcoords)
    
    # Delaunay triangulation
    tri = scipy.spatial.Delaunay(tetrahedron_points)
    print tri.simplices
    #print np.shape(targets_points)
    for ii in np.arange(np.shape(targets_points)[0]):
        point = targets_points[ii,:]
        tetrahedra = tri.find_simplex(point)
        if tetrahedra >=  0: 
            # point located in the tetrahedron
            try:
                res_bc = fun_get_barycentric_coordinates(tetrahedra, tri, point)
                print ii, res_bc
                res_bcoords[ii,:] = res_bc
            except ValueError:
                #print ii#, point, np.array([-1, -1, -1])
                res_bcoords[ii,:] = np.array([-1, -1, -1, -1])
        else:
            res_bcoords[ii,:] = np.array([-1, -1, -1, -1])# * np.ones((4,1))
            #print ii,# point, np.array([-1, -1, -1])
    return res_bcoords
    
def fun_get_barycentric_coordinates(tetrahedra, trig, point):
    """
    Here we solve the equation to get the barycentric coordinates where
    tetrahedra are the points defining the testrahedron
    point are the point coordinates in cartersian space
    trig the result of the Delaunay triangulation
    """
    # find the 
    X = trig.transform[tetrahedra,:3]
    Y = point - trig.transform[tetrahedra,3]
    b = np.einsum('ijk,ik->ij', X, Y)
    b_coords = np.c_[b, 1 - b.sum(axis=1)]
    print b_coords
    return b_coords

In [19]:
for ii in np.arange(len(list_index_tetra)):
    tetra_points = set_3D_points_A[list_index_tetra[ii,:],:]
    #print tetra_points
    bary_coords = fun_compute_barycentric_coordinates(tetra_points, set_3D_points_B)
    #print bary_coords

[[2 0 1 3]]
[[2 0 1 3]]
[[0 2 1 3]]
[[2 0 1 3]]
[[3 2 1 0]]
[[2 0 1 3]]
[[2 0 1 3]]
[[3 0 1 2]]
[[0 3 1 2]]
[[3 0 1 2]]
[[3 0 1 2]]
[[3 0 1 2]]
[[0 3 1 2]]
[[0 2 1 3]]
[[2 0 1 3]]
[[3 0 1 2]]
[[3 0 1 2]]
[[2 0 1 3]]
[[0 2 1 3]]
[[0 3 1 2]]
[[0 3 1 2]]
[[0 2 1 3]]
[[3 0 1 2]]
[[3 0 1 2]]
[[3 0 1 2]]
[[2 0 1 3]]
[[2 0 1 3]]
[[2 0 1 3]]
[[3 0 1 2]]
[[0 3 1 2]]
[[3 0 1 2]]
[[1 0 3 2]]
[[3 0 1 2]]
[[0 3 1 2]]
[[2 0 1 3]]
[[2 0 1 3]]
[[1 0 3 2]]
[[3 0 1 2]]
[[2 0 1 3]]
[[0 2 1 3]]
[[0 1 3 2]]
[[0 3 1 2]]
[[0 2 1 3]]
[[1 0 3 2]]
[[3 0 1 2]]
[[3 0 1 2]]
[[1 0 2 3]]
[[1 0 2 3]]
[[2 0 1 3]]
[[2 3 0 1]]
[[2 3 0 1]]
[[2 0 3 1]]
[[2 3 0 1]]
[[2 3 0 1]]
[[3 2 0 1]]
[[0 2 3 1]]
[[3 2 0 1]]
[[3 2 0 1]]
[[2 0 3 1]]
[[2 3 0 1]]
[[3 2 0 1]]
[[3 0 2 1]]
[[0 3 2 1]]
[[2 3 0 1]]
[[1 2 0 3]]
[[0 1 3 2]]
[[1 3 0 2]]
[[2 1 0 3]]
[[1 0 3 2]]
[[1 3 0 2]]
[[1 3 0 2]]
[[3 0 2 1]]
[[1 0 2 3]]
[[1 2 0 3]]
[[0 1 3 2]]
[[1 3 0 2]]
[[3 1 0 2]]
[[0 3 2 1]]
[[0 1 2 3]]
[[1 2 0 3]]
[[3 0 2 1]]
[[3 0 2 1]]
[[3 2 0 1]]
[[0 

In [20]:
# Delaunay triangulation
tri = scipy.spatial.Delaunay(set_3D_points_A)
print "I have %1.0f tetrahedra after the triangulation. " % np.shape(tri.simplices)[0]

I have 21 tetrahedra after the triangulation. 


In [21]:
# find which point are in which tetrahedron
tetrahedra = tri.find_simplex(set_3D_points_B)
print tetrahedra

# find the 
X = tri.transform[tetrahedra,:3]
Y = set_3D_points_B - tri.transform[tetrahedra,3]
b = np.einsum('ijk,ik->ij', X, Y)
bcoords = np.c_[b, 1 - b.sum(axis=1)]
print bcoords


[-1 10  5 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  4 -1 -1  4]
[[-2.00281506 -1.68671534  4.38478264  0.30474775]
 [ 0.12967523  0.51228443  0.33944733  0.018593  ]
 [ 0.2110668   0.43815101  0.3105616   0.04022059]
 [ 0.77781074 -1.85113009  2.8656892  -0.79236985]
 [ 1.29300663  0.45844588  0.94265938 -1.69411189]
 [-0.16872596 -0.15580732  2.1035175  -0.77898422]
 [ 1.84829194 -0.34177734  0.26152659 -0.76804118]
 [ 3.77135058  1.57762599 -2.47747858 -1.87149799]
 [-0.45021515 -0.80831455  3.84252085 -1.58399114]
 [ 0.5776942  -0.28059686  2.23045333 -1.52755068]
 [ 1.65329647  1.01284176  0.06216699 -1.72830521]
 [-0.23520829 -1.94882911  3.36406789 -0.18003049]
 [ 0.10451994 -1.2782272   2.57800032 -0.40429306]
 [-1.24854886 -1.0673117   3.58722049 -0.27135993]
 [-0.02036716 -0.89477045  2.30436156 -0.38922395]
 [ 0.80232102 -0.11907202  1.6471174  -1.3303664 ]
 [ 2.28828791  1.51816209 -1.2282402  -1.57820981]
 [ 0.31012444  0.18988767  0.1336698   0.36631809]
 [ 2.84617475  0.

In [22]:
# display the barycentric coordinate for each point in its corresponding tetrahedra
for tt in np.arange(len(tetrahedra)):
    if tetrahedra[tt] >= 0: # then the point is in a tetrahedron
        print "indexes of tetradron"
        print tri.simplices[tetrahedra[tt],:], set_3D_points_B[tt,:]
        print set_3D_points_A[tri.simplices[tetrahedra[tt],:],:]

indexes of tetradron
[2 8 4 7] [ 0.65987566  0.55350566  0.71766314]
[[ 0.85381135  0.526314    0.48443321]
 [ 0.672502    0.65261187  0.8138237 ]
 [ 0.55169608  0.42001619  0.64783663]
 [ 0.93440559  0.44960437  0.9696401 ]]
indexes of tetradron
[2 0 7 1] [ 0.85346095  0.33654726  0.7537496 ]
[[ 0.85381135  0.526314    0.48443321]
 [ 0.78291039  0.14393185  0.75492195]
 [ 0.93440559  0.44960437  0.9696401 ]
 [ 0.99516805  0.56603119  0.4872887 ]]
indexes of tetradron
[2 4 6 3] [ 0.39813471  0.66326931  0.44443213]
[[ 0.85381135  0.526314    0.48443321]
 [ 0.55169608  0.42001619  0.64783663]
 [ 0.06050081  0.63298337  0.16609815]
 [ 0.05596099  0.91636157  0.40669315]]
indexes of tetradron
[2 4 6 3] [ 0.5  0.5  0.5]
[[ 0.85381135  0.526314    0.48443321]
 [ 0.55169608  0.42001619  0.64783663]
 [ 0.06050081  0.63298337  0.16609815]
 [ 0.05596099  0.91636157  0.40669315]]
